# Welcome to TQ42

## Example of Hybrid Quantum Multilayer Perceptron (HQMLP)

In [1]:
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    AlgorithmProto, 
    ActFuncProto,
    OptimProto, 
    LossFuncProto, 
    MeasurementModeProto,
    MeasureProto,
    EntanglingProto,
    DiffMethodProto,
    TSEvalInputsProto,
    QubitTypeProto,
    DatasetStorageInfoProto, 
    TSTrainInputsProto,
    TSHQMLPTrainMetadataProto, 
    TSHQMLPTrainParametersProto
)
from google.protobuf.json_format import MessageToDict

# Create a client and selecting org, proj and exp

In [2]:

with TQ42Client() as client:
    org_sample = list_all_organizations(client=client)[0] 
    proj_sample = list_all_projects(client=client, organization_id=org_sample.id)[0]
    exp_sample = list_all_experiments(client=client, project_id=proj_sample.id)[-1]

    print(proj_sample)
    print(org_sample)
    print(exp_sample)



# Set parameters for the train

In [3]:

parameters_train = MessageToDict(TSHQMLPTrainMetadataProto(
    parameters=TSHQMLPTrainParametersProto(
        input_width=1,
        label_width=1,
        hidden_size=17,
        num_qubits=8,
        depth=7,
        measurement_mode=MeasurementModeProto.NONE,
        rotation=MeasureProto.X,
        entangling=EntanglingProto.BASIC,
        measure=MeasureProto.Z,
        diff_method=DiffMethodProto.ADJOINT,
        qubit_type=QubitTypeProto.LIGHTNING_QUBIT,
        act_func=ActFuncProto.SIGMOID,
        dropout=False,
        dropout_p=0.2,
        bn=False,
        num_epochs=5,
        batch_size=20,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
    ),
    inputs=TSTrainInputsProto(
        data=DatasetStorageInfoProto(storage_id="1fcc5e6c-b962-410b-a257-739f7172db03")
    )
), preserving_proto_field_name=True)

# Train the model

In [4]:

run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.TS_HQMLP_TRAIN, 
    experiment_id=exp_sample.id,
    compute=HardwareProto.SMALL, 
    parameters=parameters_train
)

print(run.data)

id: "544cbfac-feb8-4179-901f-8b47dde9b5c0"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 38
status: QUEUED
algorithm: TS_HQMLP_TRAIN
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"hidden_size\":17,\"num_qubits\":8,\"depth\":7,\"measurement_mode\":\"NONE\",\"rotation\":\"X\",\"entangling\":\"BASIC\",\"measure\":\"Z\",\"diff_method\":\"ADJOINT\",\"qubit_type\":\"LIGHTNING_QUBIT\",\"act_func\":\"SIGMOID\",\"dropout\":false,\"dropout_p\":0.2,\"bn\":false,\"num_epochs\":5,\"batch_size\":20,\"learning_rate\":0.05,\"optim\":\"ADAM\",\"loss_func\":\"MSE\",\"train_model_info\":null},\"inputs\":{\"data\":{\"storage_id\":\"1fcc5e6c-b962-410b-a257-739f7172db03\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705664885
  nanos: 342501208
}


# Import functions for the evaluate

In [5]:
from tq42.algorithm import (
    ModelStorageInfoProto,
    TSHQMLPEvalMetadataProto,
    TSHQMLPEvalParametersProto
) 

# Set parameters for the evaluate

In [6]:
params_eval =  MessageToDict(TSHQMLPEvalMetadataProto(
    parameters=TSHQMLPEvalParametersProto(
        input_width=1,
        label_width=1,
        hidden_size=17,
        num_qubits=8,
        depth=7,
        measurement_mode=MeasurementModeProto.NONE,
        rotation=MeasureProto.X,
        entangling=EntanglingProto.BASIC,
        measure=MeasureProto.Z,
        diff_method=DiffMethodProto.ADJOINT,
        qubit_type=QubitTypeProto.LIGHTNING_QUBIT,
        act_func=ActFuncProto.SIGMOID,
        dropout=False,
        dropout_p=0.2,
        bn=False,
    ),
    inputs=TSEvalInputsProto(
        model=ModelStorageInfoProto(storage_id="MODEL_BUCKET_STORAGE_ID"),
        data=DatasetStorageInfoProto(storage_id="DATA_BUCKET_STORAGE_ID")
    )  
), preserving_proto_field_name=True)

# Evaluate the model

In [7]:
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.TS_HQMLP_EVAL,
        experiment_id=exp_sample.id,
        compute=HardwareProto.SMALL,
        parameters=params_eval
    )
print(run.data)

id: "0b0900fb-3713-47d7-9802-60ac80d1f71b"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 39
status: QUEUED
algorithm: TS_HQMLP_EVAL
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"hidden_size\":17,\"num_qubits\":8,\"depth\":7,\"measurement_mode\":\"NONE\",\"rotation\":\"X\",\"entangling\":\"BASIC\",\"measure\":\"Z\",\"diff_method\":\"ADJOINT\",\"qubit_type\":\"LIGHTNING_QUBIT\",\"act_func\":\"SIGMOID\",\"dropout\":false,\"dropout_p\":0.2,\"bn\":false},\"inputs\":{\"model\":{\"storage_id\":\"MODEL_BUCKET_STORAGE_ID\"},\"data\":{\"storage_id\":\"DATA_BUCKET_STORAGE_ID\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705664891
  nanos: 31514947
}
